In [124]:
import torch
from finetune.cifar10_models import resnet18

## Binary Image Classification with a Pre-Trained Model

### Loading a pre-trained model

In [114]:
model_ft = resnet18(pretrained=True)
model_ft.fc

Linear(in_features=512, out_features=10, bias=True)

In [115]:
for param in model_ft.parameters():
    param.requires_grad = False

In [116]:
feature_dim = model_ft.fc.in_features
model_ft.fc = nn.Sequential()

### Preparing Data

In [117]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset, TensorDataset
import numpy as np

In [118]:
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010]
    )
])

In [119]:
# Creating a custom PyTorch dataset to create our binary dataset

class MyCIFAR10(datasets.CIFAR10):
    def __init__(self, *args, include_list=[], **kwargs):
        super(MyCIFAR10, self).__init__(*args, **kwargs)

        if include_list == []: return

        targets = np.array(self.targets)
        include = np.array(include_list).reshape(1, -1)
        mask = (targets.reshape(-1, 1) == include).any(axis=1)

        self.data = self.data[mask]
        self.targets = targets[mask].tolist()

In [120]:
include = [0,1]
train_dataset = MyCIFAR10('data', include_list=include, train=True, download=True, transform=preprocess)
val_dataset = MyCIFAR10('data', include_list=include, train=False, download=True, transform=preprocess)
label_names = ['airplane', 'automobile']

Files already downloaded and verified
Files already downloaded and verified


In [121]:
train_dataset.__len__(), val_dataset.__len__()

(10000, 2000)

In [134]:
batch_size = 8

train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

dataloaders = {'train' : train_data_loader,
              'val' : val_data_loader}

### Setting up the training loop

In [125]:
import torch.optim as optim
import torch.nn as nn

In [123]:
model = nn.Linear(feature_dim, 2)

In [126]:
num_epochs = 10
lr = 0.01
optimizer = optim.SGD(model.parameters(), lr)
criterion = nn.CrossEntropyLoss()

In [136]:
def train_model(model, model_ft, dataloaders, criterion, optimizer, num_epochs=25):

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs
                labels = labels

                # zero the parameter gradients
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):

                    features = model_ft(inputs)
                    outputs = model(features)
                    loss = criterion(outputs, labels)
                    print(loss)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

    return model

In [137]:
for name, param in model.named_parameters():
    if param.requires_grad: print(name)

weight
bias


In [ ]:
train_model(model, model_ft, dataloaders, criterion, optimizer, num_epochs)